In [ ]:
# Initialize Otter
import otter 
grader = otter.Notebook()

## Homework 2: Pandas Review

**This assignment is due Tuesday, October 14th at 11:59 PM.**

### Collaboration Policy

Data science is a collaborative activity. While you may talk with others about the labs, we ask that you **write your solutions individually**. If you do discuss the assignments with others please **include their names** below. (That's a good way to learn your classmates' names.)

**Collaborators**: *list collaborators here*

[Pandas](https://pandas.pydata.org/) is one of the most widely used Python libraries in data science. In this lab, you will review the commonly used data wrangling operations/tools in Pandas. We aim to give you familiarity with:

* Creating dataframes
* Slicing data frames (i.e. selecting rows and columns)
* Filtering data (using boolean arrays)
* Split-Apply-Combine operations with `.groupby`
* Joining data with `merge`

In this lab you are going to use several pandas methods, such as `drop` and `loc`. You may enter `help(method_name)` to see helpful documentation for that method. If you are still familar with the `datascience` library used in DSCI 101/102, the conversion reference notebook included with the assignment might serve useful. 

**Note**: The Pandas interface is notoriously confusing, and the documentation is not consistently great. Throughout the semester, you will have to search through Pandas documentation and experiment, but remember it is part of the learning experience and will help shape you as a data scientist!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

## Section 1: Creating DataFrames & Basic Manipulations

A [dataframe](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe) is a table in which each column has a type; there is an index over the columns (typically string labels) and an index over the rows (typically ordinal numbers).

The [docs](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) for the pandas `DataFrame` class  provide at least two syntaxes to create a data frame.

**Syntax 1: ** You can create a data frame by specifying the columns and values using a dictionary as shown below. 

The keys of the dictionary are the column names, and the values of the dictionary are lists containing the row entries.

In [ ]:
fruit_info = pd.DataFrame(
    data={'fruit': ['apple', 'orange', 'banana', 'raspberry'],
          'color': ['red', 'orange', 'yellow', 'pink']
          })
fruit_info

**Syntax 2: ** You can also define a dataframe by specifying the rows like below. 

Each row corresponds to a distinct tuple, and the columns are specified separately.

In [ ]:
fruit_info2 = pd.DataFrame(
    [("red", "apple"), ("orange", "orange"), ("yellow", "banana"),
     ("pink", "raspberry")], 
    columns = ["color", "fruit"])
fruit_info2

You can obtain the dimensions of a dataframe by using the shape attribute `dataframe.shape`.

In [ ]:
fruit_info.shape

You can also convert the entire dataframe into a two-dimensional numpy array.

In [ ]:
fruit_info.values

### Question 1.1

For a DataFrame `d`, you can add a column with `d['new column name'] = ...` and assign a list or array of values to the column. Add a column of integers containing 1, 2, 3, and 4 called `rank1` to the `fruit_info` table which expresses your personal preference about the taste ordering for each fruit (1 is tastiest; 4 is least tasty). 

In [ ]:
...
fruit_info

In [ ]:
grader.check("q1_1")

### Question 1.2

Use the `.drop()` method to [drop](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html) the `rank1` column you created. (Make sure to use the `axis` parameter correctly.) Note that `drop` does not change a table, but instead returns a new table with fewer columns or rows unless you set the optional `inplace` parameter.

In [ ]:
fruit_info_original = ...
fruit_info_original

In [ ]:
grader.check("q1_2")

### Question 1.3a

Use the `.rename()` method to [rename](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rename.html) the columns of `fruit_info_original` so they begin with capital letters. Set this new dataframe to `fruit_info_caps`.

In [ ]:
fruit_info_caps = ...
fruit_info_caps

In [ ]:
grader.check("q1_3a")

### Question 1.3b

Oops! When we labeled the color of apples, we forgot to mention that our apple was a Granny Smith. Use `.replace` to convert the value of red to green in the "Color" column. 

In [ ]:
...
fruit_info_caps

In [ ]:
grader.check("q1_3b")

### Babyname dataset
Now that we have learned the basics, let's move on to the babynames dataset. The babynames dataset contains a record of the given names of babies born in the United States each year.

First let's run the following cells to build the dataframe `baby_names`. The cell below builds a dataframe from a .zip file. There should be a total of 6028151 records. If written to a .csv, these data would be significantly larger than .zip file (which is already fairly large)

This cells builds the full `baby_names` DataFrame. It first builds one dataframe per state, because that's how the data are stored in the zip file. Here is documentation for [pd.concat](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.concat.html) if you want to know more about its functionality. 

In [ ]:
import zipfile
zf = zipfile.ZipFile("./data/namesbystate.zip", 'r')

column_labels = ['State', 'Sex', 'Year', 'Name', 'Count']

def load_dataframe_from_zip(zf, f):
    with zf.open(f) as fh: 
        return pd.read_csv(fh, header=None, names=column_labels)

states = [
    load_dataframe_from_zip(zf, f)
    for f in sorted(zf.filelist, key=lambda x:x.filename) 
    if f.filename.endswith('.TXT')
]

baby_names = states[0]
for state_df in states[1:]:
    baby_names = pd.concat([baby_names, state_df])
baby_names = baby_names.reset_index().iloc[:, 1:]

In [ ]:
zipfile.ZipFile(namesbystate_path, 'r')

In [ ]:
len(baby_names)

In [ ]:
baby_names.head()

## Slicing Data Frames - selecting rows and columns


### Selection Using Label/Index (using loc)

**Column Selection** 

To select a column of a `DataFrame` by column label, the safest and fastest way is to use the `.loc` [method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.loc.html). General usage of `.loc` looks like `df.loc[rowname, colname]`. (Reminder that the colon `:` means "everything.")  For example, if we want the `color` column of the `ex` data frame, we would use: `ex.loc[:, 'color']`

- You can also slice across columns. For example, `baby_names.loc[:, 'Name':]` would select the column `Name` and all columns after `Name`.

- *Alternative:* While `.loc` is invaluable when writing production code (it is fast), it may be a little too verbose for interactive use. One recommended alternative is the `[]` method, which takes on the form `df['colname']`.

**Row Selection**

Similarly, if we want to select a row by its label, we can use the same `.loc` method. In this case, the "label" of each row refers to the index (ie. primary key) of the dataframe.

In [ ]:
#Example:
baby_names.loc[2:5, 'Name']

In [ ]:
#Example:  Notice the difference between these two methods
#Just passing in 'Name' returns a Series while ['Name'] returns a Dataframe
baby_names.loc[2:5, ['Name']]

The `.loc` actually uses the Pandas row index rather than row id/position of rows in the dataframe to perform the selection. Also, notice that if you write `2:5` with `loc[]`, contrary to normal Python slicing functionality, the end index is included, so you get the row with index 5. 


### Selection using Integer location (using iloc)

`iloc[]` lets you slice the dataframe by row position and column position instead of by row index and column label (which is the case for `loc[]`). This is really the main difference between the 2 functions and it is **important** that you remember the difference and why you might want to use one over the other. In addition, with `iloc[]`, the end index is NOT included, like with normal Python slicing.

As a mnemonic, remember that the i in `iloc` means "integer". 

Below, we have sorted the `baby_names` dataframe. Notice how the *position* of a row is not necessarily equal to the *index* of a row. For example, the first row is not necessarily the row associated with index 1. This distinction is important in understanding the different between `loc[]` and `iloc[]`.

In [ ]:
sorted_baby_names = baby_names.sort_values(by=['Name'])
sorted_baby_names.head()

Here is an example of how we would get the 2nd, 3rd, and 4th rows with only the `Name` column of the `baby_names` dataframe using both `iloc[]` and `loc[]`. Observe the difference, especially after sorting `baby_names` by name.

In [ ]:
sorted_baby_names.iloc[1:4, 3]

Notice that using `loc[]` with 1:4 gives different results, since it selects using the *index*.

In [ ]:
sorted_baby_names.loc[1:4, "Name"]

Lastly, we can change the index of a dataframe using the `set_index` method.

In [ ]:
#Example: We change the index from 0,1,2... to the Name column
df = baby_names[:5].set_index("Name") 
df

We can now lookup rows by name directly:

In [ ]:
df.loc[['Mary', 'Anna'], :]

However, if we still want to access rows by location we will need to use the integer loc (`iloc`) accessor:

In [ ]:
#Example: 
#df.loc[2:5,"Year"] You can't do this
df.iloc[1:4, 2:3]

### Question 1.4

Selecting multiple columns is easy.  You just need to supply a list of column names.  Select the `Name` and `Year` **in that order** from the `baby_names` table.

In [ ]:
name_and_year = ...

In [ ]:
grader.check("q1_4")

Note that `.loc[]` can be used to re-order the columns within a dataframe.

## Filtering Data

### Filtering with boolean arrays

Filtering is the process of removing unwanted material.  In your quest for cleaner data, you will undoubtedly filter your data at some point: whether it be for clearing up cases with missing values, for culling out fishy outliers, or for analyzing subgroups of your data set.  Note that compound expressions have to be grouped with parentheses. Example usage looks like `df[df['column name'] < 5]]`.

For your reference, some commonly used comparison operators are given below.

Symbol | Usage      | Meaning 
------ | ---------- | -------------------------------------
==   | a == b   | Does a equal b?
<=   | a <= b   | Is a less than or equal to b?
&#62;=   | a >= b   | Is a greater than or equal to b?
<    | a < b    | Is a less than b?
&#62;    | a &#62; b    | Is a greater than b?
~    | ~p       | Returns negation of p
&#124; | p &#124; q | p OR q
&    | p & q    | p AND q
^  | p ^ q | p XOR q (exclusive or)

In the following we construct the DataFrame containing only names registered in California

In [ ]:
ca = baby_names[baby_names['State'] == 'CA']

### Question 1.5
Using a boolean array, select the names in Year 2000 (from `baby_names`) that have larger than 3000 counts. Keep all columns from the original `baby_names` dataframe.

Note: Any time you use `p & q` to filter the dataframe, make sure to use `df[(df[p]) & (df[q])]` or `df.loc[(df[p]) & (df[q])]`. That is, make sure to wrap conditions with parentheses.

**Remember** that both slicing and `loc` will achieve the same result, it is just that the former is a little more idiomatic (readable) while the latter `loc` is typically faster in production. You are free to use whichever one you would like.

In [ ]:
result = ...
result.head()

In [ ]:
grader.check("q1_5")

Optionally, repeat the exercise from above, but this time using the `query` command.

In [ ]:
result_using_query = ...
result_using_query.head(5)

### Question 1.6

Some names gain/lose popularity because of cultural phenomena such as a famous pop star. Below, we plot the popularity of the name Britney in California over time. What do you notice about this plot? What might be the cause of the steep spike around the year 2000?

In [ ]:
britney_baby_name = baby_names[(baby_names['Name'] == 'Britney') & (baby_names['State'] == 'CA') & (baby_names['Sex'] == 'F')]
plt.plot(britney_baby_name['Year'], britney_baby_name['Count'])
plt.title("Britney Popularity Over Time")
plt.xlabel('Year')
plt.ylabel('Count');

**Your Response:** ...

### Question 1.7

Now we will aggregate counts across all 50 states. Use `groupby()` to aggregate the data by "Name" and "Year", then `sum()` "Count" and finally `reset_index()` (this will turn the index, Name in this case, into a column).  

In [ ]:
counts_aggregated_by_name_and_year = ...

In [ ]:
grader.check("q1_7")

Now you can use the below code to plot the frequency of any name of interest in the United States over time. 

In [ ]:
name_of_interest = 'name_of_interest'
chosen_baby_name = counts_aggregated_by_name_and_year.query("Name == @name_of_interest")
plt.plot(chosen_baby_name['Year'], chosen_baby_name['Count'])
plt.title(f"Popularity Of {name_of_interest} Over Time")
plt.xlabel('Year')
plt.ylabel('Count');

Well done. Now that you've been primed on data manipulation with `pandas`, we're going to do a little more practice without as much hand holding to challenge you. 

## Section 2: Filtering and aggregating COVID data

In thi section we'll examine the influence of the COVID pandemic on US jobs across different sectors.

In [ ]:
jobs = pd.read_csv("./data/jobs.csv")
jobs

#### Question 2.1

Let's start by creating a new variable called "difference" in "jobs" that is the available labor force subtracted from jobs. Make this as a new column in your data.

In [ ]:
...
jobs.head()

In [ ]:
grader.check("q2_1")

What do positive and negative differences mean? 

**Your response**: A positive difference means there are more jobs than people available to take them, and vice versa for a negative difference.

#### Question 2.2

For how many months in our time series did the number of jobs exceed the size of the labor force?

In [ ]:
months = ...
months

In [ ]:
grader.check("q2_2")

#### Question 2.3

Find the date that corresponds to the greatest negative difference between number of jobs and size of the labor force.

In [ ]:
...

Was there something going on at this point in time?

In [ ]:
sns.lineplot(x = jobs["observation_date"], y = jobs["difference"])
plt.xticks(rotation=30);

**Your response**: ...

#### Question 2.4 

Now we'll look at some data related to corporate profits before and after COVID.

In [ ]:
profits = pd.read_csv("./data/corp_profits.csv")
profits["DATE"] = pd.to_datetime(profits["DATE"])
profits.head()

What is the date range in our data? Note that we converted "DATE" to a datetime. "datetimes" can be operated on like any numeric variable. 

In [ ]:
earliest = ...
latest = ...

In [ ]:
grader.check("q2_4")

#### Question 2.5 

How were different industries affected by COVID? First, filter that data such that we only have dates beginning on Jan 1st in 2016 and after. Then create a new column in the data called "COVID" and give the value "pre-covid" if the dates are between 2016-01-01 and 2020-03-01, and "post-covid" for anything after. 

In [ ]:
profits_recent = ...
profits_recent

In [ ]:
...

In [ ]:
grader.check("q2_5")

#### Questeion 2.6

Now find the average pre- and post-covid profit for each product. Remember that you can group by more than one variable.

In [ ]:
...

Interpret the findings. Why do you think some industries saw increased profits while others saw decreased profits?

**Your response**: ...

## Section 3: Cleaning and joining data

We'll end this homework with an exercise in data cleaning and joining multiple relational datasets.

We have two datasets:
1. A dataset of every COVID treatment center in each incorporated township/city in the US
2. A dataset of the estimated population in each incorporated township/city in the US as of 2013.

Our ultimate goal is to determine which cities had the best COVID treatment infrastructure, represented by COVID treatment centers per capita. 

In [ ]:
# COVID treatment centers
treat_centers = pd.read_csv("./data/COVID-19_Treatments_20240927.csv")
# Lowercase the town/city names to canonicalize them with the next dataset.
treat_centers["City"] = treat_centers["City"].str.lower()
treat_centers.head()

#### Question 3.1

Find the number of COVID treatment centers in each town. Remember that each row is a single treatment center, and remember that towns/cities in different states can have the same name. 

*Hint*: The `size()` function will be helpful

In [ ]:
treat_counts = ...
treat_counts

In [ ]:
grader.check("q3_1")

#### Question 3.2

Let's read in the population data.

In [ ]:
# Town/city populations
city_pop = pd.read_csv("./data/pop_est.csv", encoding='latin-1')
city_pop["NAME"] = city_pop["NAME"].str.lower()
city_pop.head()

For whatever reason, the population data has many duplicated rows. First, determine how many rows are duplicated. Second, remove those rows from the data.

The easiest way to do this is with the `duplicated` method. The following cell will call up its documentation. 

In [ ]:
help(pd.DataFrame.duplicated)

In [ ]:
...

In [ ]:
city_pop = ...
city_pop

In [ ]:
grader.check("q3_2")

#### Question 3.3

We're getting close to joining the treatment centers dataframes and the population dataframe on a shared key. We might be tempted to join them on city names but this would be a careless thing to do (remember that different states can have towns/cities with the same name). This is instead a situation where multiple variables combine to form a unique key: town/city name and state. 

This poses an issue for us. While we have states in both frames, they are formatted differently. Thus, to fully relate them we need to pull in a third relation. 

In [ ]:
states = pd.read_csv("./data/States.csv")
states.head()

Join states on city_pop such that there is one new column in city_pop called "Postal". You may need to `.drop()` a redundant column after joining.

In [ ]:
city_pop = ...
city_pop

In [ ]:
grader.check("q3_3")

#### Question 3.4

Now join city_pop and treat_counts. Ensure that you're only keeping rows where there are key matches i.e. do an "inner" join. 

In [ ]:
pop_treat_merge = ...
pop_treat_merge

In [ ]:
grader.check("q3_4")

#### Question 3.5

To determine the number of treatment centers per capita, we'll need to divide center count by population. There are certain values of one or the other that could cause issues when calculating this quantity. Use the `describe()` method to summarize the quantitative columns in the merged data. 

In [ ]:
# Describe
...

If you identify any issues, filter the data to remove those values. Then create a new column called "centers_per_cap" in pop_treat_merge, representing the number of centers per capita in each town/city.

In [ ]:
# Correct any issues if relevant
...

In [ ]:
...
pop_treat_merge

In [ ]:
grader.check("q3_5")

#### Question 3.6

Identify the five cities with the highest number of treatment cities per capita.

In [ ]:
...

Do you think your findings are valid? Why or why not?

**Your response**: ...

#### Question 3.7

Filter the data for only population estimates above or equal to 1000, then again identify the top 5 cities. Return the answer as an array or series. 

In [ ]:
pop_treat_merge = ...

In [ ]:
top_cities = ...
top_cities

In [ ]:
grader.check("q3_7")

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order. Then execute the following two commands from the File menu:

 - Save and Checkpoint

 - Close and Halt

Then upload your .ipynb file to Canvas assignment Homework 2